In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('raw_dataV2.csv')

# Clean column names
data.columns = data.columns.str.strip()

# Clean string data in all columns
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].astype(str).str.strip()  # Convert to string and remove leading/trailing spaces
    data[col] = data[col].apply(lambda x: " ".join(x.split()))  # Replace multiple spaces with a single space only if x is a string

# Create a full context column so we can extract more information using NLP
data['context'] = data[['Title', 'Description', 'Attributes', 'Building Name']].astype(str).agg(', '.join, axis=1)

# Modify the 'Rooms' and 'Number of Bathrooms' columns with a regex that accounts for potential non-matches
data['Rooms'] = data['Rooms'].astype(str).str.extract(r'(\d+)').astype(float)
data['Number of Bathrooms'] = data['Number of Bathrooms'].astype(str).str.extract(r'(\d+(?:\.\d+)?)', expand=False).astype(float)

# Fill blanks in 'Den Included' with 0
data['Den Included'] = data['Den Included'].replace(np.nan, 0)

def clean_and_convert_size(size):
    # Convert size to string to safely use string methods
    size_str = str(size)
    
    # Replace "Not Available" and blanks with NaN
    if pd.isna(size) or size_str.strip() in ["", "Not Available", "nan"]:
        return np.nan
    
    # Remove commas and " sq ft", "Sq Ft", "sqft", "Ft2" for uniformity
    size_cleaned = size_str.replace(",", "").lower()
    for non_numeric in [" sq ft", "sqft", "ft2", "sq ft"]:
        size_cleaned = size_cleaned.replace(non_numeric, "")
    
    # Handle ranges by calculating the average
    if '-' in size_cleaned:
        size_range = size_cleaned.split('-')
        try:
            size = int(np.mean([int(s.strip()) for s in size_range]))
        except ValueError:
            return np.nan
    else:
        try:
            size = int(size_cleaned.strip())
        except ValueError:
            return np.nan
    
    return size


# Apply the cleaning function to the "Size (sqft)" column
data['Size (sqft)'] = data['Size (sqft)'].apply(clean_and_convert_size)

# Impute the data for Size column
data['Size (sqft)'] = pd.to_numeric(data['Size (sqft)'], errors='coerce')  # Convert to numeric, invalid parsing to NaN
data.loc[data['Size (sqft)'] < 50, 'Size (sqft)'] = np.nan  # Set values less than 50 to NaN
avg_size_per_room = data.groupby('Rooms')['Size (sqft)'].mean()
data['Size (sqft)'] = data.apply(
    lambda row: avg_size_per_room[row['Rooms']] if pd.isna(row['Size (sqft)']) else row['Size (sqft)'],
    axis=1
)
data['Size (sqft)'] = data['Size (sqft)'].astype(int)

# Handle missing values 
def handle_bathrooms(row):
    if row["Number of Bathrooms"] == "":
        if row['Rooms'] == 1:
            row["Number of Bathrooms"] = 1
        else:
            row["Number of Bathrooms"] = row['Rooms']//2
    return row
data['Number of Bathrooms'] = data['Number of Bathrooms'].fillna("")
data = data.apply(lambda row: handle_bathrooms(row), axis=1)

data.drop(['URL', 'Title', 'Location', 'Description', 'Attributes', 'Parking Included', 'Utilities Included', 'Pet Friendly', 'Furnished', 'Building Name', 'Distance from Hospital', 'Distance from Police Station', 'Distance to Nearest Store', 'Distance to Nearest Pharmacy', 'context'], axis=1, inplace=True)
data = data.dropna(subset=['Latitude', 'Longitude'])

# Write file for easier viewing
data.to_csv('cleaned_data.csv', index=False)
data.head(5)

,Price,Property Type,Rooms,Den Included,Number of Bathrooms,Latitude,Longitude,Size (sqft),Walk Score,Transit Score,Bike Score,Time to Nearest Hospital,Time to Nearest Police Station,Time to Nearest Store,Time to Nearest Pharmacy
0,3200,House,2.0,0.0,1.0,44.727257,-63.653230,1107,89.0,NaN,16.0,1004.0,1091.0,634.0,609.0
1,1829,Apartment,1.0,0.0,1.0,44.686904,-63.580551,723,89.0,65.0,80.0,591.0,673.0,284.0,182.0
2,3995,House,3.0,0.0,3.0,44.641301,-63.630122,4400,54.0,49.0,56.0,641.0,584.0,313.0,272.0
3,2500,House,3.0,1.0,3.0,44.785733,-63.715050,2500,89.0,65.0,80.0,1507.0,1594.0,876.0,870.0
4,1612,Apartment,1.0,0.0,1.0,44.636393,-63.571632,634,89.0,65.0,80.0,246.0,164.0,92.0,102.0
